<a href="https://colab.research.google.com/github/curiosity806/2020_dacon_satellite_precipitation/blob/augment8%2Fresnet-single/ResNet_base_model_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp '/content/drive/My Drive/2020 Kaggle Study/data/Copy of train.npy' train.npy

In [3]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization, Activation, concatenate, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
import warnings
import gc
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import stats
warnings.filterwarnings("ignore")
# style 설정
plt.style.use('dark_background')
plt.style.use('seaborn-colorblind')

# 재생산성을 위해 시드 고정
np.random.seed(7)
random.seed(7)
tf.random.set_seed(7)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## 데이터 받아오기

In [0]:
#train = np.load('train.npy')
train = np.load('train.npy')

# 데이터 만들기

In [0]:
train.shape

(76345, 40, 40, 15)

In [0]:
def cutoff(data):
    index = 0
    rain_index = []
    for file in data:
        target= file[:,:,-1].reshape(40,40,1)
        cutoff_labels = np.where(target < 0, 0, target)
        if (cutoff_labels > 0).sum() < 50:
            index = index + 1
            continue
        rain_index.append(index)
        index = index + 1
    return data[rain_index], rain_index

def re_landtype(data):
    rain = data[:,:,:,9].reshape(-1)
    rain = rain//100
    #data[data[:,:,:,9] == 0] = 0 #Ocean
    rain[rain == 1] = 1 #land
    rain[rain == 2] = 0.5 #costal
    rain[rain == 3] = 0.2 #in land water
    data[:,:,:,9] = rain.reshape(-1,40,40)
    return data
    
def select_ch(data,ch_list = [0,1,2,3,4,5,6,7,8,9,14]):#9: land_type, 14: precipitation
    return(data[:,:,:,ch_list])

In [0]:
train = select_ch(train)
train, rain_index = cutoff(train)
train = re_landtype(train)

#train = tf.convert_to_tensor(train, dtype=np.float32)
#train = train.numpy()
#gc.collect()

#모델 만들기

In [0]:
def create_model():
    inputs=Input((40,40,10))
    ###cnn
    bn=BatchNormalization()(inputs)
    conv0=Conv2D(256, kernel_size=1, strides=1, padding='same', activation='relu')(bn)
    
    bn=BatchNormalization()(conv0)
    conv=Conv2D(128, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
    concat=concatenate([conv0, conv], axis=3)
    
    bn=BatchNormalization()(concat)
    conv=Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
    concat=concatenate([concat, conv], axis=3)
        
    for i in range(5):
        bn=BatchNormalization()(concat)
        conv=Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
        concat=concatenate([concat, conv], axis=3)
    
    bn=BatchNormalization()(concat)
    outputs=Conv2D(1, kernel_size=1, strides=1, padding='same', activation='relu')(bn)
    
    model=Model(inputs=inputs, outputs=outputs)
    
    return model

In [0]:
from sklearn.metrics import f1_score

def mae(y_true, y_pred) :
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    over_threshold = y_true >= 0.1
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    remove_NAs = y_true >= 0
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

def maeOverFscore_keras(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

## 데이터 제네레이터 만들기

In [0]:
def recall(y_true, y_pred):
    y_true = K.ones_like(y_true) 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    all_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    
    recall = true_positives / (all_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    y_true = K.ones_like(y_true) 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
### Metric은 tf.keras.metrics.Metric이거 상속받아서 custom할 것
#아직 만드는 중....

class maeOverFscore(tf.keras.metrics.Metric):
    def __init__(self, name='multi_label_metric'):
        super(maeOverFscore, self).__init__(name=name)
        # 계산에 필요한 변수들 선언
        self.maeOverFscore = self.add_weight(name='maeOverFscore', initializer='zeros')
        #self.mae = self.add_weight(name='mae', initializer='zeros')
        #self.Fscore = self.add_weight(name='Fscore', initializer='zeros')
        
    def update_state(self, y, pred):
    	'''metric 계산'''
        TP = tf.count_nonzero(pred * y)
        TN = tf.count_nonzero((pred - 1) * (y - 1))
        FP = tf.count_nonzero(pred * (y - 1))
        FN = tf.count_nonzero((pred - 1) * y)

        precision = tf.divide(TP, TP + FP)
        precision = tf.divide(TP, TP + FN)
        Fscore =  tf.divide(2 * precision * recall, (precision + recall))
        
        threshold = 0.1

    	# prediction과 label이 완전히 일치하는 경우 계산
        correct_pred = tf.math.equal(tf.round(y), tf.round(pred))
        correct_pred = tf.reduce_sum(tf.cast(tf.reduce_all(correct_pred, 1), tf.float32))
        
    	# 변수에 더해줌
        self.corr.assign_add(correct_pred)  # 지금까지 y, prediction 완전 일치하는 횟수
        self.example_num.assign_add(y.shape[0])  # 전체 (y, prediction) 쌍의 수
    
    def result(self):
    	'''지금까지 계산된 metric 반환'''
        return self.corr / self.example_num
    
    def reset_states(self):
    	'''변수 초기화'''
        self.corr.assign(0)
        self.example_num.assign(0)

class maeOverFscore(tf.keras.metrics.Metric):

    def __init__(self, name='maeOverFscore', **kwargs):
      super(maeOverFscore, self).__init__(name=name, **kwargs)
      self.true_positives = self.add_weight(name='tp', initializer='zeros')
      self.true_positives = self.add_weight(name='tp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
      y_pred = tf.reshape(tf.argmax(y_pred, axis=1), shape=(-1, 1))
      values = tf.cast(y_true, 'int32') == tf.cast(y_pred, 'int32')
      values = tf.cast(values, 'float32')
      if sample_weight is not None:
        sample_weight = tf.cast(sample_weight, 'float32')
        values = tf.multiply(values, sample_weight)
      self.true_positives.assign_add(tf.reduce_sum(values))

    def result(self):
      return self.true_positives

    def reset_states(self):
      # The state of the metric will be reset at the start of each epoch.
      self.true_positives.assign(0.)


In [0]:
# TPU설정 안될때 실행
error = False
if error:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [49]:
import tensorflow as tf
import os
import tensorflow_datasets as tfds
from sklearn.model_selection import KFold
AUTOTUNE = tf.data.experimental.AUTOTUNE
AUTO = tf.data.experimental.AUTOTUNE

def rotate90(x,y):
    return tf.image.rot90(x), tf.image.rot90(y)

def transpose(x,y):
    return tf.image.transpose(x), tf.image.transpose(y)


def train_model(x_data, y_data, k, s):
    k_fold = KFold(n_splits=k, shuffle=True, random_state=7777)
    model_number = 0
    for train_idx, val_idx in k_fold.split(x_data):
        if model_number == s:
            x_train, y_train = x_data[train_idx], y_data[train_idx]
            x_val, y_val = x_data[val_idx], y_data[val_idx]
            
            # TPU에 모델 올리기
            resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
            tf.config.experimental_connect_to_cluster(resolver)
            # This is the TPU initialization code that has to be at the beginning.
            tf.tpu.experimental.initialize_tpu_system(resolver)
            strategy = tf.distribute.experimental.TPUStrategy(resolver)

            with strategy.scope():
                model = create_model()
                model.compile(optimizer = 'adam',loss = 'mae'#,
                #metrics=[maeOverFscore,fscore]
                )

                callbacks_list = [
                    tf.keras.callbacks.ReduceLROnPlateau(
                        monitor='val_loss',
                        patience=3,
                        factor=0.8
                    ),
                    tf.keras.callbacks.ModelCheckpoint(
                        filepath = '/content/drive/My Drive/2020 Kaggle Study/deagwon/best_model'+str(model_number)+'.h5',
                        monitor='val_fscore',
                        save_best_only=True
                    )
                ]
                

            batch_size = 128

            # batch_dataset 만들기
            non_augmented_train_batches = (
                tf.data.Dataset.from_tensor_slices((x_train, y_train))
                .repeat()
                .cache()
                .batch(batch_size = batch_size, drop_remainder=True)
                .map(rotate90 , num_parallel_calls=AUTO)
            )
            # augmentation_batches 만들기
            rotate90_train_batches = non_augmented_train_batches.map(rotate90 , num_parallel_calls=AUTO)
            rotate180_train_batches = rotate90_train_batches.map(rotate90 , num_parallel_calls=AUTO)
            rotate270_train_batches = rotate180_train_batches.map(rotate90 , num_parallel_calls=AUTO)
            train_transpose_batches = non_augmented_train_batches.map(transpose , num_parallel_calls=AUTO)
            rotate90_transpose_batches = rotate90_train_batches.map(transpose , num_parallel_calls=AUTO)
            rotate180_transpose_batches = rotate180_train_batches.map(transpose , num_parallel_calls=AUTO)
            rotate270_transpose_batches = rotate270_train_batches.map(transpose , num_parallel_calls=AUTO)
            
            # augmentation batch 합쳐서 train batches 만들기
            train_batches = non_augmented_train_batches.concatenate(rotate90_train_batches)
            train_batches = train_batches.concatenate(rotate180_train_batches)
            train_batches = train_batches.concatenate(rotate270_train_batches)
            train_batches = train_batches.concatenate(train_transpose_batches)
            train_batches = train_batches.concatenate(rotate180_transpose_batches)
            train_batches = train_batches.concatenate(rotate270_transpose_batches)

            #agumentation batch shuffle 
            train_batches = train_batches.shuffle(buffer_size = batch_size * 8)
            train_batches = train_batches.prefetch(AUTOTUNE)


            model.fit(train_batches, epochs=1,
                    steps_per_epoch = len(x_train)*8 // batch_size,
                    validation_data=(x_val, y_val))#, callbacks=callbacks_list)

            model.save(f'/content/drive/My Drive/2020 Kaggle Study/deagwon/model_trainonly_{model_number}.h5')
            break##### 일단은 한 폴드만 학습
        model_number+=1
from tensorflow.keras.models import load_model, Model
k = 5
models = []
train_model(train[:,:,:,:-1], train[:,:,:,-1].reshape(-1,40,40,1), k=k, s=4)

INFO:tensorflow:Initializing the TPU system: grpc://10.61.27.98:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.61.27.98:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


1538/1538 [==============================] - 94s 61ms/step - loss: 5.0219 - val_loss: 4.7552


In [0]:
model.save('model.h5')

In [0]:
# 훈련에서 사용하는 0~8번 채널, ladtype 만을 뽑아서 X_test에 저장하기
test_submit = np.load('/content/drive/My Drive/2020 Kaggle Study/data/test.npy')
test_submit = test_submit[:,:,:,:10]
del test_submit
gc.collect()
pred = model.predict(test_submit)

## submission 만들기

In [0]:
submission = pd.read_csv('/content/drive/My Drive/2020 Kaggle Study/data/sample_submission.csv')

In [0]:
submission.iloc[:,1:] = pred.reshape(-1, 1600)

In [0]:
submission.to_csv('Dacon_landtype.csv', index = False)

In [0]:
# 제출파일 구글 드라이브에 올리기
!cp Dacon_landtype.csv '/content/drive/My Drive/2020 Kaggle Study/submission'